# ARTIFICIAL INTELLIGENCE ALGORITHM PROJECT

# OPTIMIZATION OF BISCUIT PRODUCTION USING AI ALGORITHMS

# Team Members
1. Rubesh PRABAKARAN
2. Devaraj RAMAMOORTHY
3. Sethulakshmi KOCHCHIRAYIL BABU

# PROBLEM DESCRIPTION

1. A biscuit manufacturing factory is planning to produce a series of biscuits for Christmas.
2. Their aim is to produce of various biscuits of different sizes and shapes using same size of dough.
3. The goal is to maximize the biscuit production from a single roll and ensuring highest profit parallelly.

# Defined Constraints

Following are the constraints in the problem:
1. No overlapping of biscuits on the dough roll
2. Each biscuit should have tolerance for defects, and shouldn't violoate defect threshold
3. Any part of dough that is unused for biscuits shall be applied with penalties of –1 per position
4. For ensuring that the solution strictly follows these constraints, we intend to make functions that :
5. Validation of no biscuit placement overlap
6. Validation of non-violation of defect thresholds
7. Tracking of unused positions for penalty calculations


# Solution Conditions

1. Biscuits must be placed at integer positions
2. Overlapping of biscuits is not allowed
3. Total size of the biscuit should not exceed the length of the dough roll.
4. The penalty for unused positions is calculated as:<br/>
	**Penalty=−1×(Number of unused positions)**

# Algorithms to Solve the Problem

The key to successfully solving the problem is to choose an algorithm that can find the best arrangement of biscuits while satisfying the constraints.<br/>
Two potential approaches are:<br/><br/>
     * Constraint Satisfaction Problem (CSP)<br/>
     * Genetic Algorithm (GA)


# Implementation of Algorithm

Importing the packages

In [28]:
import pandas as pd
import numpy as np

Loading the dataset into 'defects' variable

In [29]:
# Load defect data
defects = pd.read_csv('/content/defects.csv')

# Inspect the columns and data to dynamically adapt
print("Columns in defects.csv:", defects.columns)
print(defects.head())

Columns in defects.csv: Index(['x', 'class'], dtype='object')
            x class
0  355.449335     c
1   92.496236     a
2  141.876795     c
3  431.833902     c
4  435.028461     c


Inserting the values from dataset into Dictionary

In [30]:
# Dynamically handle column names for 'position' and 'class'
position_col = defects.columns[0]  # Assuming 'position' is the first column
class_col = defects.columns[1]    # Assuming 'class' is the second column

# Convert defects into a usable structure
defects_dict = {}
for _, row in defects.iterrows():
    position = row[position_col]
    defect_type = row[class_col]
    if position not in defects_dict:
        defects_dict[position] = {}
    defects_dict[position][defect_type] = defects_dict[position].get(defect_type, 0) + 1

# Print the structured defects dictionary
print("Structured Defects Dictionary:")
print(defects_dict)

Structured Defects Dictionary:
{355.44933495113816: {'c': 1}, 92.49623624899968: {'a': 1}, 141.87679475713: {'c': 1}, 431.833901528095: {'c': 1}, 435.02846071222535: {'c': 1}, 205.80069498110916: {'a': 1}, 34.68761156005529: {'b': 1}, 443.5663536577713: {'a': 1}, 69.42321664429218: {'a': 1}, 301.2810691469357: {'a': 1}, 342.65158129802404: {'c': 1}, 216.1188466276893: {'a': 1}, 367.6738857180436: {'c': 1}, 93.12612310082602: {'a': 1}, 401.42223511282566: {'c': 1}, 74.24620676846693: {'c': 1}, 356.8414017746271: {'a': 1}, 457.419132049754: {'a': 1}, 395.64337374148624: {'c': 1}, 368.8939083383807: {'a': 1}, 163.9165054130055: {'a': 1}, 484.1606702022739: {'b': 1}, 55.26603588274765: {'c': 1}, 171.55890817166824: {'b': 1}, 119.691651582648: {'a': 1}, 332.6392861077555: {'b': 1}, 85.10413501075348: {'a': 1}, 289.67861866874034: {'b': 1}, 156.79452239165255: {'b': 1}, 143.5296603398663: {'c': 1}, 83.61463228838583: {'a': 1}, 108.52309512246006: {'a': 1}, 57.5774716792582: {'c': 1}, 9.95619

## Constraint Satisfaction Problems (CSP)

In [31]:
# Function to calculate defects within a range
def calculate_defects(start, length):
    defects_count = {'a': 0, 'b': 0, 'c': 0}
    for pos in range(start, start + length):
        if pos in defects_dict:
            for defect_type in defects_dict[pos]:
                defects_count[defect_type] += defects_dict[pos][defect_type]
    return defects_count

In [32]:
# Function to validate biscuit placement
def is_valid(start, biscuit):
    defects_in_range = calculate_defects(start, biscuit['length'])
    for defect_type, max_allowed in biscuit['max_defects'].items():
        if defects_in_range.get(defect_type, 0) > max_allowed:
            return False
    return True

In [33]:
# Manual CSP implementation
def optimize_biscuits():
    best_score = float('-inf')
    best_arrangement = None
    ROLL_LENGTH = 500
    biscuits= {
    0: {'length': 4, 'value': 3, 'max_defects': {'a': 4, 'b': 2, 'c': 3}},
    1: {'length': 8, 'value': 12, 'max_defects': {'a': 5, 'b': 4, 'c': 4}},
    2: {'length': 2, 'value': 1, 'max_defects': {'a': 1, 'b': 2, 'c': 1}},
    3: {'length': 5, 'value': 8, 'max_defects': {'a': 2, 'b': 3, 'c': 2}},
    }

    # Generate all possible arrangements
    for start in range(ROLL_LENGTH):
        arrangement = []
        score = 0
        used_positions = set()

        for pos in range(start, ROLL_LENGTH):
            for biscuit_id, biscuit in biscuits.items():
                if pos + biscuit['length'] <= ROLL_LENGTH and pos not in used_positions:
                    if is_valid(pos, biscuit):
                        arrangement.append((pos, biscuit_id))
                        score += biscuit['value']
                        used_positions.update(range(pos, pos + biscuit['length']))
                        break  # Move to next position

        # Apply penalty for unused positions
        penalty = -(ROLL_LENGTH - len(used_positions))
        score += penalty

        # Update the best arrangement
        if score > best_score:
            best_score = score
            best_arrangement = arrangement
            best_penalty = penalty

    return best_arrangement, best_score, best_penalty

# Run the optimization
arrangement, score, penalty  = optimize_biscuits()

# Print the results
print("Best Arrangement:", arrangement)
print("Best Score:", score)
print("Penalty: ",penalty)


Best Arrangement: [(0, 0), (4, 0), (8, 0), (12, 0), (16, 0), (20, 0), (24, 0), (28, 0), (32, 0), (36, 0), (40, 0), (44, 0), (48, 0), (52, 0), (56, 0), (60, 0), (64, 0), (68, 0), (72, 0), (76, 0), (80, 0), (84, 0), (88, 0), (92, 0), (96, 0), (100, 0), (104, 0), (108, 0), (112, 0), (116, 0), (120, 0), (124, 0), (128, 0), (132, 0), (136, 0), (140, 0), (144, 0), (148, 0), (152, 0), (156, 0), (160, 0), (164, 0), (168, 0), (172, 0), (176, 0), (180, 0), (184, 0), (188, 0), (192, 0), (196, 0), (200, 0), (204, 0), (208, 0), (212, 0), (216, 0), (220, 0), (224, 0), (228, 0), (232, 0), (236, 0), (240, 0), (244, 0), (248, 0), (252, 0), (256, 0), (260, 0), (264, 0), (268, 0), (272, 0), (276, 0), (280, 0), (284, 0), (288, 0), (292, 0), (296, 0), (300, 0), (304, 0), (308, 0), (312, 0), (316, 0), (320, 0), (324, 0), (328, 0), (332, 0), (336, 0), (340, 0), (344, 0), (348, 0), (352, 0), (356, 0), (360, 0), (364, 0), (368, 0), (372, 0), (376, 0), (380, 0), (384, 0), (388, 0), (392, 0), (396, 0), (400, 0),

## Explanation of the Contraint Satisfaction Problem (CSP) Algorithm implementation

This code solves a problem where biscuits are placed on a dough roll of fixed length while ensuring certain constraints are satisfied. The goal is to maximize the value of the biscuits placed while considering defects and penalties for unused positions. Here's an in-depth breakdown of each step and function:

1. Loading Defect Data

        defects = pd.read_csv('/content/sample_data/defects.csv')
This reads the defect data from a CSV file using pandas read_csv function.
defects will be a DataFrame containing the defect information, where each row represents a defect at a specific position on the roll.
2. Parameters and Biscuit Information

        ROLL_LENGTH = 500
        biscuits = {
        0: {'length': 4, 'value': 3, 'max_defects': {'a': 4, 'b': 2, 'c': 3}},
        1: {'length': 8, 'value': 12, 'max_defects': {'a': 5, 'b': 4, 'c': 4}},
        2: {'length': 2, 'value': 1, 'max_defects': {'a': 1, 'b': 2, 'c': 1}},
        3: {'length': 5, 'value': 8, 'max_defects': {'a': 2, 'b': 3, 'c': 2}},
        }
ROLL_LENGTH defines the total length of the dough roll.
biscuits is a dictionary containing information about the biscuits, such as:
length: The size of the biscuit (how much space it occupies on the roll).
value: The value of the biscuit (how much it contributes to the score).
max_defects: A dictionary of allowed defects per biscuit type ('a', 'b', 'c'), indicating how many defects of each type can occur within the biscuit's length.
3. Converting Defects into a Usable Structure<br/>

        defects_dict = {}
        for _, row in defects.iterrows():
          position = int(np.floor(row['x']))
          defect_type = row['class']
          if position not in defects_dict:
            defects_dict[position] = {}
            defects_dict[position][defect_type] = defects_dict[position].get(defect_type, 0) + 1

  This code converts the defect data from the CSV into a dictionary format (defects_dict).
For each defect, the position (integer part of the 'x' coordinate) is used as the key, and the defect type ('class') is used to count how many defects of each type occur at each position on the roll.
defects_dict will look like {position: {defect_type: count}}.
4. Precomputing Cumulative Defects

        cumulative_defects = {'a': [0] * ROLL_LENGTH, 'b': [0] * ROLL_LENGTH, 'c': [0] * ROLL_LENGTH}

        for pos in range(ROLL_LENGTH):
          for defect_type in cumulative_defects:
            if pos > 0:
              cumulative_defects[defect_type][pos] = cumulative_defects[defect_type][pos - 1]
            if pos in defects_dict and defect_type in defects_dict[pos]:
              cumulative_defects[defect_type][pos] += defects_dict[pos][defect_type]
cumulative_defects is a dictionary that stores the cumulative number of defects for each defect type ('a', 'b', 'c') up to each position on the roll.
For each position on the roll, it adds up the defects for each type from the previous position.
This allows quick access to the total number of defects within any range on the roll without recalculating it repeatedly.
5. Optimized Defect Calculation <br/>

        def calculate_defects_optimized(start, length):
        defects_count = {}
          for defect_type in cumulative_defects:
            end = min(start + length - 1, ROLL_LENGTH - 1)
            defects_count[defect_type] = cumulative_defects[defect_type][end]
            if start > 0:
                defects_count[defect_type] -= cumulative_defects[defect_type][start - 1]
        return defects_count
This function calculates the number of defects in a given range of the roll (start to start + length).
It uses the precomputed cumulative_defects to quickly find the number of defects for each defect type in the range.
This avoids iterating over the defects repeatedly and speeds up the process.
6. Valid Placement Check
        def is_valid(start, biscuit):
        defects_in_range = calculate_defects_optimized(start, biscuit['length'])
        for defect_type, max_allowed in biscuit['max_defects'].items():
          if defects_in_range.get(defect_type, 0) > max_allowed:
            return False
        return True
This function checks whether placing a biscuit at a given position (start) is valid.
It calculates the number of defects in the range covered by the biscuit using calculate_defects_optimized.
It then compares the defects with the allowed maximum defects for each type. If any defect type exceeds the threshold, the placement is invalid.
7. Sorting Biscuits by Value-to-Length Ratio <br/>

        sorted_biscuits = sorted(biscuits.items(), key=lambda x: -x[1]['value'] / x[1]['length'])
This sorts the biscuits in descending order based on their value-to-length ratio.
The higher the ratio, the better the biscuit is in terms of value relative to its size. This ensures that the biscuits with the best value are considered first for placement.
8. Dynamic Programming (DP) Approach with Penalty

        dp = [-float('inf')] * (ROLL_LENGTH + 1)
        dp[ROLL_LENGTH] = 0  # Base case: no roll left means no score
This initializes the DP table (dp), where dp[pos] will store the maximum score that can be obtained from position pos to the end of the roll.
dp[ROLL_LENGTH] = 0 is the base case, meaning if there is no roll left, the score is zero.
9. DP Iteration Over Positions

        for pos in range(ROLL_LENGTH - 1, -1, -1):
          dp[pos] = dp[pos + 1] - 1  # Subtract 1 for unused position penalty
          placements[pos] = placements[pos + 1][:]  # Copy previous placement
The outer loop iterates over each position on the roll in reverse order (from ROLL_LENGTH - 1 to 0).
For each position, it assumes that the position is left unused and incurs a penalty of -1 (wasted space).
The previous placement (placements[pos + 1]) is copied to maintain the record of placements for the next iteration.
10. Placing Biscuits

        for biscuit_id, biscuit in sorted_biscuits:
          end_pos = pos + biscuit['length']
          if end_pos <= ROLL_LENGTH and is_valid(pos, biscuit):  # Check if placement is valid
            score_with_biscuit = biscuit['value'] + dp[end_pos]
            if score_with_biscuit > dp[pos]:
              dp[pos] = score_with_biscuit
              placements[pos] = [(biscuit_id, pos)] + placements[end_pos]
  For each valid biscuit, the code checks whether placing it at the current position is feasible (does not exceed the roll length, and satisfies defect constraints).
If placing the biscuit results in a higher score, it updates the dp value at the current position and records the placement.
11. Extracting and Displaying the Best Arrangement

        best_score = dp[0]
        best_arrangement = placements[0]

12. Display the results
        print("Best Arrangement:")
        for biscuit_id, position in best_arrangement:
          print(f"Biscuit {biscuit_id} placed at position {position}")
          print(f"\nBest Score (after penalty): {best_score}")
After the loop, dp[0] contains the best possible score, starting from the first position.
placements[0] contains the optimal arrangement of biscuits.
The best arrangement and score are then printed, including positions of biscuits and the final score considering the penalties for unused positions.
Summary:
The code implements a dynamic programming approach with a penalty for unused positions, considering defect constraints for each biscuit placement. It efficiently calculates the maximum value arrangement while adhering to the problem's rules and constraints, ensuring that the score is maximized while considering the penalty for empty spaces.

## Integrating Constraints Satisfaction Problems (CSP) with the Dynamic Programming (DP) approach

We are integrating the *Dynamic Programming* approach with the *Constraint Satisfactory Problems* algorithm to increace the efficiency positioning  arragement of the biscuits.

In [34]:
# Parameters
ROLL_LENGTH = 500
biscuits = {
    0: {'length': 4, 'value': 3, 'max_defects': {'a': 4, 'b': 2, 'c': 3}},
    1: {'length': 8, 'value': 12, 'max_defects': {'a': 5, 'b': 4, 'c': 4}},
    2: {'length': 2, 'value': 1, 'max_defects': {'a': 1, 'b': 2, 'c': 1}},
    3: {'length': 5, 'value': 8, 'max_defects': {'a': 2, 'b': 3, 'c': 2}},
}

In [35]:
# Convert defects into a usable structure
defects_dict = {}
for _, row in defects.iterrows():
    position = int(np.floor(row['x']))
    defect_type = row['class']
    if position not in defects_dict:
        defects_dict[position] = {}
    defects_dict[position][defect_type] = defects_dict[position].get(defect_type, 0) + 1

In [36]:
# Precompute cumulative defects for quick lookup
cumulative_defects = {'a': [0] * ROLL_LENGTH, 'b': [0] * ROLL_LENGTH, 'c': [0] * ROLL_LENGTH}

for pos in range(ROLL_LENGTH):
    for defect_type in cumulative_defects:
        if pos > 0:
            cumulative_defects[defect_type][pos] = cumulative_defects[defect_type][pos - 1]
        if pos in defects_dict and defect_type in defects_dict[pos]:
            cumulative_defects[defect_type][pos] += defects_dict[pos][defect_type]

In [37]:
# Function to calculate defects within a range (optimized)
def calculate_defects_optimized(start, length):
    defects_count = {}
    for defect_type in cumulative_defects:
        end = min(start + length - 1, ROLL_LENGTH - 1)
        defects_count[defect_type] = cumulative_defects[defect_type][end]
        if start > 0:
            defects_count[defect_type] -= cumulative_defects[defect_type][start - 1]
    return defects_count

In [38]:
# Function to check if placing a biscuit is valid
def is_valid(start, biscuit):
    defects_in_range = calculate_defects_optimized(start, biscuit['length'])
    for defect_type, max_allowed in biscuit['max_defects'].items():
        if defects_in_range.get(defect_type, 0) > max_allowed:
            return False
    return True

In [39]:
# Sort biscuits by value-to-length ratio
sorted_biscuits = sorted(
    biscuits.items(), key=lambda x: -x[1]['value'] / x[1]['length']
)

In [40]:
# Dynamic Programming (DP) approach with penalty for unused positions
dp = [-float('inf')] * (ROLL_LENGTH + 1)
dp[ROLL_LENGTH] = 0  # Base case: no roll left means no score

# Track placements for reconstruction
placements = [[] for _ in range(ROLL_LENGTH + 1)]

In [41]:
for pos in range(ROLL_LENGTH - 1, -1, -1):
    # Default: skip this position and incur a penalty
    dp[pos] = dp[pos + 1] - 1  # Subtract 1 for unused position penalty
    placements[pos] = placements[pos + 1][:]  # Copy previous placement

    for biscuit_id, biscuit in sorted_biscuits:
        end_pos = pos + biscuit['length']
        if end_pos <= ROLL_LENGTH and is_valid(pos, biscuit):  # Check if placement is valid
            score_with_biscuit = biscuit['value'] + dp[end_pos]
            if score_with_biscuit > dp[pos]:
                dp[pos] = score_with_biscuit
                placements[pos] = [(biscuit_id, pos)] + placements[end_pos]

# Extracting the best arrangement and score
best_score = dp[0]
best_arrangement = placements[0]

In [42]:
# Display the results
print("Best Arrangement:")
for biscuit_id, position in best_arrangement:
    print(f"Biscuit {biscuit_id} placed at position {position}")
print(f"\nBest Score (after penalty): {best_score}")

Best Arrangement:
Biscuit 1 placed at position 0
Biscuit 1 placed at position 8
Biscuit 3 placed at position 16
Biscuit 1 placed at position 21
Biscuit 3 placed at position 29
Biscuit 2 placed at position 34
Biscuit 0 placed at position 36
Biscuit 2 placed at position 41
Biscuit 1 placed at position 43
Biscuit 3 placed at position 52
Biscuit 1 placed at position 57
Biscuit 1 placed at position 65
Biscuit 1 placed at position 73
Biscuit 1 placed at position 81
Biscuit 1 placed at position 89
Biscuit 1 placed at position 98
Biscuit 2 placed at position 106
Biscuit 3 placed at position 108
Biscuit 0 placed at position 114
Biscuit 1 placed at position 119
Biscuit 3 placed at position 127
Biscuit 1 placed at position 132
Biscuit 0 placed at position 140
Biscuit 3 placed at position 144
Biscuit 1 placed at position 149
Biscuit 1 placed at position 158
Biscuit 1 placed at position 168
Biscuit 1 placed at position 176
Biscuit 2 placed at position 185
Biscuit 1 placed at position 187
Biscuit 3 

## Explanation for the Constraint Satisfication Problems (CSP) with Dynamic Programming Approach Implementation

The step by step explanation of the code for solving the problem using Constraint Satisfication (CSP) with Dynamic Programming (DP), which includes the penalty for unused positions.

1. Parameters and Biscuit Information

        ROLL_LENGTH = 500
        biscuits = {
        0: {'length': 4, 'value': 3, 'max_defects': {'a': 4, 'b': 2, 'c': 3}},
        1: {'length': 8, 'value': 12, 'max_defects': {'a': 5, 'b': 4, 'c': 4}},
        2: {'length': 2, 'value': 1, 'max_defects': {'a': 1, 'b': 2, 'c': 1}},
        3: {'length': 5, 'value': 8, 'max_defects': {'a': 2, 'b': 3, 'c': 2}},
        }
ROLL_LENGTH defines the total length of the dough roll.
biscuits is a dictionary containing information about the biscuits, such as:
length: The size of the biscuit (how much space it occupies on the roll).
value: The value of the biscuit (how much it contributes to the score).
max_defects: A dictionary of allowed defects per biscuit type ('a', 'b', 'c'), indicating how many defects of each type can occur within the biscuit's length.
2. Converting Defects into a Usable Structure<br/>

        defects_dict = {}
        for _, row in defects.iterrows():
          position = int(np.floor(row['x']))
          defect_type = row['class']
          if position not in defects_dict:
            defects_dict[position] = {}
            defects_dict[position][defect_type] = defects_dict[position].get(defect_type, 0) + 1

  This code converts the defect data from the CSV into a dictionary format (defects_dict).
For each defect, the position (integer part of the 'x' coordinate) is used as the key, and the defect type ('class') is used to count how many defects of each type occur at each position on the roll.
defects_dict will look like {position: {defect_type: count}}.
3. Precompute Cumulative Defects for Quick Lookup

        cumulative_defects = {'a': [0] * ROLL_LENGTH, 'b': [0] * ROLL_LENGTH, 'c': [0] * ROLL_LENGTH}

        for pos in range(ROLL_LENGTH):
          for defect_type in cumulative_defects:
            if pos > 0:
              cumulative_defects[defect_type][pos] = cumulative_defects[defect_type][pos - 1]
            if pos in defects_dict and defect_type in defects_dict[pos]:
              cumulative_defects[defect_type][pos] += defects_dict[pos][defect_type]

  It calculates the cumulative sum of each type of defect across the roll for efficient range queries. First, the code initializes the cumulative_defects dictionary with lists of zeros for each defect type, which will represent positions from 0 to ROLL_LENGTH - 1. The algorithm then goes through every position in the roll and updates the cumulative sums by adding the counts from the defects_dict at each position. If a position doesn't have any defects, then the cumulative sums just copy the values from the previous position. This preprocessing step allows calculating the total number of defects for an arbitrary range in constant time: it is sufficient to subtract the cumulative sums at the range boundaries.

4. Calculate Defects in a Range (Optimized)

        def calculate_defects_optimized(start, length):
            defects_count = {}
            for defect_type in cumulative_defects:
              end = min(start + length - 1, ROLL_LENGTH - 1)
              defects_count[defect_type] = cumulative_defects[defect_type][end]
              if start > 0:
                  defects_count[defect_type] -= cumulative_defects[defect_type][start - 1]
        return defects_count

  The calculate_defects_optimized function makes use of precomputed cumulative_defects in order to calculate defect counts across a range. It takes a start position and the length of a range. For each type of defect, it calculates how many defects are cumulatively up to the position after the end of the range, subtracting the sum over positions before the start of the range, if any. In this way, calculation over ranges is far superior compared to iterating over all the defects within a particular range.

5. Check if Placing a Biscuit is Valid

        def is_valid(start, biscuit):
          defects_in_range = calculate_defects_optimized(start, biscuit['length'])
          for defect_type, max_allowed in biscuit['max_defects'].items():
            if defects_in_range.get(defect_type, 0) > max_allowed:
              return False
          return True

  The is_valid function checks if a biscuit can be placed at a certain starting position without violating its defect constraints. It calls calculate_defects_optimized to get the defect counts within the range the biscuit occupies. It then compares these counts with the maximum number of defects allowed for each type by the biscuit (max_defects). If any defect count exceeds the maximum value allowed, the placement is considered invalid, and this function returns False. Otherwise, it returns True. This function ensures that biscuit placements meet the problem constraints.

6. Sort Biscuits by Value-to-Length Ratio

        sorted_biscuits = sorted( biscuits.items(), key=lambda x: -x[1]['value'] / x[1]['length'])

  The code sorts the biscuits in descending order according to their value-to-length ratio to prioritize them for placing. This heuristic, here using Python's sorted with a lambda expression, calculates for every biscuit the ratio of its value to its length and performs a sort based on it. This prioritization strategy therefore tries to maximize the score by preferring biscuits providing a high value relative to their size.

7. Dynamic Programming (DP) Setup

        dp = [-float('inf')] * (ROLL_LENGTH + 1)
        dp[ROLL_LENGTH] = 0

        placements = [[] for _ in range(ROLL_LENGTH + 1)]

  The core of the optimization in this dynamic programming approach starts with initializing a dp array. It has -inf in all positions, except for the last position-ROLL_LENGTH set to 0 for the base case. This is a representation of the maximum score that can be achieved starting from each position, with no score possible when the roll is exhausted. Also, an auxiliary placement list is initialized to store, for each entry, the sequence of biscuits for the optimal solution starting at that position.

8. DP Transition Logic

        for pos in range(ROLL_LENGTH - 1, -1, -1):
          dp[pos] = dp[pos + 1] - 1  # Subtract 1 for unused position penalty
          placements[pos] = placements[pos + 1][:]  # Copy previous placement
            for biscuit_id, biscuit in sorted_biscuits:
              end_pos = pos + biscuit['length']
              if end_pos <= ROLL_LENGTH and is_valid(pos, biscuit):  # Check if placement is valid
                  score_with_biscuit = biscuit['value'] + dp[end_pos]
                  if score_with_biscuit > dp[pos]:
                      dp[pos] = score_with_biscuit
                      placements[pos] = [(biscuit_id, pos)] + placements[end_pos]

  The DP logic iterates in reverse over all positions on the roll, considering each position as a possible start. By default, there is a penalty of -1 for skipping a position to decrease the score when a position is not used. It examines each biscuit at each position. If a biscuit can fit within the remaining roll and meets the defect constraints, the algorithm calculates the potential score as the biscuit's value added to the score at the end of the biscuit's range: dp[end_pos]. In case this score is greater than the maximum score on dp[pos], it updates dp[] and placements. This would ensure that the solution goes through all valid placements and their penalties for an optimum score.

9. Extract Results

        best_score = dp[0]
        best_arrangement = placements[0]

  After the computation of DP, the best score and best arrangement of biscuits are extracted from the dp array and placements list, respectively. The best_score represents the maximum achievable score from the very first position by considering all the constraints and penalties.

10. Display Results

        print("Best Arrangement:")
        for biscuit_id, position in best_arrangement:
            print(f"Biscuit {biscuit_id} placed at position {position}")
        print(f"\nBest Score (after penalty): {best_score}")

  The final section prints the optimal arrangement of biscuits and their respective starting positions, followed by the best score after penalties. This output summarizes the results of the optimization, providing both the solution and its effectiveness in achieving the problem's objectives.

## Genetic Algorithm

In [43]:
import random

In [44]:
# Parameters
ROLL_LENGTH = 500
biscuits = {
    0: {'length': 4, 'value': 3, 'max_defects': {'a': 4, 'b': 2, 'c': 3}},
    1: {'length': 8, 'value': 12, 'max_defects': {'a': 5, 'b': 4, 'c': 4}},
    2: {'length': 2, 'value': 1, 'max_defects': {'a': 1, 'b': 2, 'c': 1}},
    3: {'length': 5, 'value': 8, 'max_defects': {'a': 2, 'b': 3, 'c': 2}},
}

In [45]:
# Convert defects into a usable structure
defects_dict = {}
for _, row in defects.iterrows():
    position = int(np.floor(row['x']))
    defect_type = row['class']
    if position not in defects_dict:
        defects_dict[position] = {}
    defects_dict[position][defect_type] = defects_dict[position].get(defect_type, 0) + 1

In [46]:
# Precompute cumulative defects for quick lookup
cumulative_defects = {'a': [0] * ROLL_LENGTH, 'b': [0] * ROLL_LENGTH, 'c': [0] * ROLL_LENGTH}

for pos in range(ROLL_LENGTH):
    for defect_type in cumulative_defects:
        if pos > 0:
            cumulative_defects[defect_type][pos] = cumulative_defects[defect_type][pos - 1]
        if pos in defects_dict and defect_type in defects_dict[pos]:
            cumulative_defects[defect_type][pos] += defects_dict[pos][defect_type]


In [47]:
# Function to calculate defects within a range (optimized)
def calculate_defects_optimized(start, length):
    defects_count = {}
    for defect_type in cumulative_defects:
        end = min(start + length - 1, ROLL_LENGTH - 1)
        defects_count[defect_type] = cumulative_defects[defect_type][end]
        if start > 0:
            defects_count[defect_type] -= cumulative_defects[defect_type][start - 1]
    return defects_count

In [48]:
# Function to check if placing a biscuit is valid
def is_valid(start, biscuit):
    defects_in_range = calculate_defects_optimized(start, biscuit['length'])
    for defect_type, max_allowed in biscuit['max_defects'].items():
        if defects_in_range.get(defect_type, 0) > max_allowed:
            return False
    return True

In [49]:
# Generate initial population (random solutions)
def generate_individual():
    individual = [-1] * ROLL_LENGTH  # -1 means no biscuit placed at this position
    for biscuit_id, biscuit in biscuits.items():
        pos = random.randint(0, ROLL_LENGTH - biscuit['length'])
        if is_valid(pos, biscuit):
            for i in range(biscuit['length']):
                individual[pos + i] = biscuit_id
    return individual

In [50]:
# Fitness function to evaluate the score
def fitness(individual):
    score = 0
    penalty = 0
    for i in range(ROLL_LENGTH):
        if individual[i] != -1:
            biscuit = biscuits[individual[i]]
            score += biscuit['value']
        else:
            penalty -= 1  # Apply penalty for unused positions

    return score + penalty

In [51]:
# Tournament selection to pick two parents
def select_parents(population):
    parents = random.sample(population, 3)
    parents.sort(key=lambda x: fitness(x), reverse=True)
    return parents[0], parents[1]

In [52]:
# Crossover function (single-point crossover)
def crossover(parent1, parent2):
    crossover_point = random.randint(1, ROLL_LENGTH - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

In [53]:
# Mutation function
def mutate(individual, mutation_rate=0.1):
    if random.random() < mutation_rate:
        pos = random.randint(0, ROLL_LENGTH - 1)
        biscuit_id = random.choice(list(biscuits.keys()))
        biscuit = biscuits[biscuit_id]

        # If placing the biscuit at pos is valid, mutate
        if is_valid(pos, biscuit):
            individual[pos] = biscuit_id
    return individual

In [54]:
# Main Genetic Algorithm
def genetic_algorithm(population_size=50, generations=100, mutation_rate=0.1):
    # Initialize population
    population = [generate_individual() for _ in range(population_size)]

    best_individual = None
    best_fitness = -float('inf')

    # Evolution process
    for gen in range(generations):
        new_population = []
        for _ in range(population_size // 2):  # Create pairs of parents
            parent1, parent2 = select_parents(population)
            child1, child2 = crossover(parent1, parent2)

            # Apply mutation
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.append(child1)
            new_population.append(child2)

        # Replace old population with new population
        population = new_population

        # Evaluate the population
        for individual in population:
            ind_fitness = fitness(individual)
            if ind_fitness > best_fitness:
                best_fitness = ind_fitness
                best_individual = individual

        print(f"Generation {gen + 1}: Best Fitness = {best_fitness}")

    return best_individual, best_fitness

In [55]:
# Run the genetic algorithm
best_individual, best_fitness = genetic_algorithm()

# Display the results
print("\nBest Arrangement:")
for i in range(ROLL_LENGTH):
    if best_individual[i] != -1:
        print(f"Biscuit {best_individual[i]} placed at position {i}")
print(f"\nBest Score (after penalty): {best_fitness}")


Generation 1: Best Fitness = -211
Generation 2: Best Fitness = -119
Generation 3: Best Fitness = -107
Generation 4: Best Fitness = -82
Generation 5: Best Fitness = -73
Generation 6: Best Fitness = -36
Generation 7: Best Fitness = 44
Generation 8: Best Fitness = 44
Generation 9: Best Fitness = 68
Generation 10: Best Fitness = 82
Generation 11: Best Fitness = 160
Generation 12: Best Fitness = 160
Generation 13: Best Fitness = 160
Generation 14: Best Fitness = 220
Generation 15: Best Fitness = 283
Generation 16: Best Fitness = 283
Generation 17: Best Fitness = 401
Generation 18: Best Fitness = 401
Generation 19: Best Fitness = 401
Generation 20: Best Fitness = 401
Generation 21: Best Fitness = 403
Generation 22: Best Fitness = 403
Generation 23: Best Fitness = 403
Generation 24: Best Fitness = 407
Generation 25: Best Fitness = 442
Generation 26: Best Fitness = 442
Generation 27: Best Fitness = 444
Generation 28: Best Fitness = 453
Generation 29: Best Fitness = 453
Generation 30: Best Fitn

## Explanation for the Genetic Algorithm (GA) Implementation

Here is a step-by-step explanation of the code for solving the problem using Genetic Algorithms (GA), which includes the penalty for unused positions.

1. Importing Required Libraries

        import random
pandas: Used to read and handle the defect data (if available in a CSV format).
numpy: Used for numerical operations, like manipulating arrays and generating random numbers.
random: Used for randomization, such as creating initial population and mutation in the GA.
2. Loading Defect Data

        defects = pd.read_csv('/content/sample_data/defects.csv')
This code loads the defect data from a CSV file (defects.csv). It expects the file to have columns for positions (x) and defect types (class).
3. Setting Parameters

        ROLL_LENGTH = 500
        biscuits = {
        0: {'length': 4, 'value': 3, 'max_defects': {'a': 4, 'b': 2, 'c': 3}},
        1: {'length': 8, 'value': 12, 'max_defects': {'a': 5, 'b': 4, 'c': 4}},
        2: {'length': 2, 'value': 1, 'max_defects': {'a': 1, 'b': 2, 'c': 1}},
        3: {'length': 5, 'value': 8, 'max_defects': {'a': 2, 'b': 3, 'c': 2}},
        }
ROLL_LENGTH: This is the length of the dough roll (500 positions).
biscuits: A dictionary containing information about each biscuit, including:
length: The size of the biscuit.
value: The value of the biscuit.
max_defects: The maximum allowed defects for each defect type ('a', 'b', and 'c') in the biscuit's range.
4. Converting Defect Data into Usable Format

        defects_dict = {}
        for _, row in defects.iterrows():
          position = int(np.floor(row['x']))
          defect_type = row['class']
          if position not in defects_dict:
            defects_dict[position] = {}
            defects_dict[position][defect_type] = defects_dict[position].get(defect_type, 0) + 1
defects_dict: This dictionary stores the defects at each position on the dough roll.
Each position in the dough roll (position) maps to a dictionary of defect types.
The defect count for each defect type ('a', 'b', 'c') is accumulated at each position.
5. Precomputing Cumulative Defects

        cumulative_defects = {'a': [0] * ROLL_LENGTH, 'b': [0] * ROLL_LENGTH, 'c': [0] * ROLL_LENGTH}

        for pos in range(ROLL_LENGTH):
          for defect_type in cumulative_defects:
            if pos > 0:
              cumulative_defects[defect_type][pos] = cumulative_defects[defect_type][pos - 1]
              if pos in defects_dict and defect_type in defects_dict[pos]:
                cumulative_defects[defect_type][pos] += defects_dict[pos][defect_type]
cumulative_defects: This array holds the cumulative number of defects up to each position in the dough roll.
For each defect type ('a', 'b', 'c'), it accumulates the defects from the start up to each position, allowing us to quickly calculate defects in any given range later.
6. Function to Calculate Defects in a Range
        def calculate_defects_optimized(start, length):
        defects_count = {}
        for defect_type in cumulative_defects:
          end = min(start + length - 1, ROLL_LENGTH - 1)
          defects_count[defect_type] = cumulative_defects[defect_type][end]
          if start > 0:
            defects_count[defect_type] -= cumulative_defects[defect_type][start - 1]
        return defects_count
calculate_defects_optimized: This function calculates the number of defects for each defect type in a given range from start to start + length.
It uses the cumulative_defects array for efficient range queries, avoiding the need to recompute the defect count every time.
7. Function to Check If a Biscuit Placement is Valid

        def is_valid(start, biscuit):
        defects_in_range = calculate_defects_optimized(start, biscuit['length'])
        for defect_type, max_allowed in biscuit['max_defects'].items():
          if defects_in_range.get(defect_type, 0) > max_allowed:
            return False
        return True
is_valid: This function checks if a biscuit can be placed at a given position (start).
It ensures that the number of defects of each type in the range covered by the biscuit does not exceed the allowed threshold (max_defects).
8. Fitness Function for Genetic Algorithm

        def fitness(individual):
        score = 0
        penalty = 0
        for i in range(ROLL_LENGTH):
          if individual[i] != -1:
            biscuit = biscuits[individual[i]]
            score += biscuit['value']
          else:
            penalty -= 1
        return score + penalty
fitness: This function calculates the fitness of an individual solution (a placement of biscuits on the dough roll).
For each position, it checks if a biscuit is placed (individual[i] != -1):
If a biscuit is placed, its value is added to the score.
If no biscuit is placed (individual[i] == -1), a penalty of -1 is applied for that position.
The final fitness is the sum of the biscuit values and the penalties for unused positions.
9. Initialization of Population

        def create_individual():
        individual = [-1] * ROLL_LENGTH
        for biscuit_id, biscuit in biscuits.items():
          position = random.randint(0, ROLL_LENGTH - biscuit['length'])
          if is_valid(position, biscuit):
            for i in range(position, position + biscuit['length']):
                individual[i] = biscuit_id
        return individual
create_individual: This function generates a random individual (a solution) by placing biscuits randomly on the dough roll.
It tries to place each biscuit at a random position, ensuring that the placement is valid (checked using the is_valid function).
If the placement is valid, the biscuit ID is assigned to the corresponding positions in the dough roll (individual[i] = biscuit_id).
10. Mutation Function

        def mutate(individual):
        idx = random.randint(0, ROLL_LENGTH - 1)
        if individual[idx] != -1:
          biscuit_id = individual[idx]
          biscuit = biscuits[biscuit_id]
          new_position = random.randint(0, ROLL_LENGTH - biscuit['length'])
          if is_valid(new_position, biscuit):
            for i in range(idx, idx + biscuit['length']):
                individual[i] = -1
              for i in range(new_position, new_position + biscuit['length']):
                individual[i] = biscuit_id
mutate: This function introduces a mutation in an individual by randomly changing the position of one biscuit.
It selects a biscuit in the individual and attempts to place it at a new valid position.
If the placement is valid, the biscuit is moved to the new position.
11. Genetic Algorithm Setup

        population_size = 100
        generations = 200
        mutation_rate = 0.05
population_size: The number of individuals in the population.
generations: The number of generations to run the algorithm.
mutation_rate: The probability of mutation for each individual.
12. Main Genetic Algorithm Loop

        population = [create_individual() for _ in range(population_size)]

        for generation in range(generations):
            population = sorted(population, key=fitness, reverse=True)  # Sort individuals by fitness
            new_population = population[:10]  # Keep the top 10 individuals
    
            while len(new_population) < population_size:
              parent1 = random.choice(population[:50])
              parent2 = random.choice(population[:50])
              child = crossover(parent1, parent2)
        
              if random.random() < mutation_rate:
                mutate(child)
        
        new_population.append(child)
    
        population = new_population
  **Population:** Initializes the population with random individuals.
for generation in range(generations): Runs the genetic algorithm for a specified number of generations.
The population is sorted by fitness (high to low), and the top individuals are selected to form the next generation.
Parents are selected from the top 50 individuals to create offspring via crossover and mutation.
13. Displaying Results

        best_individual = max(population, key=fitness)
        print("Best Solution:", best_individual)
        print("Best Fitness (with penalty):", fitness(best_individual))
After running the algorithm for the specified generations, the best individual in the population (highest fitness) is selected.
The best individual and its fitness (including penalties for unused positions) are printed as the final solution.
Conclusion:
This code uses Genetic Algorithms (GA) to solve the problem of biscuit placement on a dough roll while considering the constraints on defects and minimizing unused positions (penalty). The fitness function incorporates penalties for unused positions, which drives the algorithm to find better solutions that minimize waste while maximizing the value of placed biscuits.

# Results and Evaluation

The results of the two algorithms are given below:

*   Constraints Satisfactory Problems with Dynamic Programming : **715**
*   Genetic Algorithm : **771**

Based the above results, the **Genetic Algorithm** performs well than the Constraints Satisfactory Problems with Dynamic Programming Algorithm by providing the highest results

1. The **Genetic Algorithm** is more effective in optimizing the constraints and finding better solutions within the problem space.
2. **Constraints Satisfactory Problems with Dynamic Programming**, while satisfactory, may have limitations in handling the complexity or adaptability required by this specific problem.


# Conclusion

The **Genetic Algorithm** should be the preferred choice for solving this biscuit production problem, especially when solution quality and optimization are constraints. However, further analysis could consider factors like computational efficiency, time complexity, and the scalability of each algorithm to ensure the best overall decision.